In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from dataclasses import dataclass
from typing import Callable, List

from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, clone

import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

TRAIN_CSV = '/kaggle/input/playground-series-s3e20/train.csv'
TEST_CSV = '/kaggle/input/playground-series-s3e20/test.csv'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Base pipeline classes

In [2]:
from abc import ABC, abstractmethod

class BaseAlgorithm(ABC):
    @abstractmethod
    def fit(self, train_df:pd.DataFrame): pass

    
    @abstractmethod
    def predict(self, test_df:pd.DataFrame): pass

    
    def fit_predict(self, train_df:pd.DataFrame, test_df:pd.DataFrame):
        self.fit(train_df)
        return self.predict(test_df)

    

@dataclass
class BasePipeline(ABC):
    train_df: pd.DataFrame
    test_df: pd.DataFrame
    target: str
        
    metrics: Callable = None
    algorithm: BaseAlgorithm = None
    
    def _clear_splits(self):
        self.train_cv = []
        self.val_cv = []
        self.gt_cv = []


    def _add_split(self, train_df, val_df, val_gt):
        self.train_cv.append(train_df)
        self.val_cv.append(val_df)
        self.gt_cv.append(val_gt)        


    @abstractmethod
    def prepare_cv(self): pass            
            
        
    def cv(self, use_train=False):
        train_results = []
        val_results = []
        for (train_df, val_df, val_gt) in zip(self.train_cv, self.val_cv, self.gt_cv):
            val_prediction = self.algorithm.fit_predict(train_df, val_df)
            val_results.append(self.metrics(val_gt, val_prediction))

            if use_train:
                train_prediction = self.algorithm.predict(train_df.drop(self.target, axis=1))
                train_results.append(self.metrics(train_df[self.target], train_prediction))
        mean = np.mean(val_results)
        std = np.std(val_results)
        if use_train:
            print(f'Train results={train_results}\tVal results={val_results}\tMean = {mean:.2f} ± {std:.2f}')
        else:
            print(f'Val results={val_results}\tMean = {mean:.2f} ± {std:.2f}')

        
    def predict(self):
        return self.algorithm.fit_predict(self.train_df, self.test_df)

# Preprocessor

In [3]:
class Preprocessor:
    proc_col = '__csv_n'
    def __init__(self, csvs):
        dfs = []
        self.df_n = len(csvs)
        for c, csv in enumerate(csvs):
            df = pd.read_csv(csv)
            df[self.proc_col] = c
            dfs.append(df)
        
        self.df = pd.concat(dfs)
        self.add_place_id()
        self.add_date_column()


    def get_dfs(self):
        return [self.df[self.df[self.proc_col] == n].drop(self.proc_col, axis=1) for n in range(self.df_n)]


    def add_place_id(self, preserve_place_str=False):
        coordinates = [str(row[0]) + '_' + str(row[1]) for _, row in self.df[['latitude', 'longitude']].drop_duplicates().iterrows()]
        coords = dict(zip(coordinates, range(len(coordinates))))

        self.df['place'] = self.df.latitude.astype(str) + '_' + self.df.longitude.astype(str)
        self.df['pid'] = self.df.place.map(coords)
        self.coords = np.array([tuple(row) for _, row in self.df[['latitude', 'longitude']].drop_duplicates().iterrows()])
        
        if not preserve_place_str:
            self.df.drop('place', axis=1, inplace=True)


    def add_date_column(self):
        self.df['date'] = self.df.year + self.df.week_no / 100
        self.df['month'] = self.df[['year', 'week_no']].apply(lambda row: datetime.datetime.strptime(f'{row["year"]}-{row["week_no"]+1}-1', "%Y-%W-%w").month, axis=1)
        self.df['is_covid'] = (self.df['year'] == 2020) & (self.df['month'] > 2) | (self.df['year'] == 2021) & (self.df['month'] == 1)
        self.df['is_lockdown'] = (self.df['year'] == 2020) & ((self.df['month'].isin([3,4])))        

# Pipeline

In [4]:
class Pipeline(BasePipeline):
    # This defines how CV is organized
    # Particularly, we create 3-fold CV based on 3 available years
    def prepare_cv(self):
        self._clear_splits()
        for year in range(2019, 2022):
            train_df = self.train_df[(self.train_df.year != year)].copy()
            val_df = self.train_df[self.train_df.year == year].copy()
            val_gt = val_df[self.target]
            val_df.drop(self.target, axis=1, inplace=True)
            self._add_split(train_df, val_df, val_gt)
            
class Pipeline2021(BasePipeline):
    # This defines how CV is organized
    # Particularly, we create 3-fold CV based on 3 available years
    def prepare_cv(self):
        self._clear_splits()
        for train_ys, val_ys in [((2019, ), (2020, )), ((2020, ), (2021, )), ((2019, ), (2020, 2021)), ((2019, 2020), (2021, )), ]:
            train_df = self.train_df[self.train_df.year.isin(train_ys)].copy()
            val_df = self.train_df[self.train_df.year.isin(val_ys)].copy()
            val_gt = val_df[self.target]
            val_df.drop(self.target, axis=1, inplace=True)
            self._add_split(train_df, val_df, val_gt)          

# PCA1 Algorithm

In [5]:
if False:
    @dataclass
    class PCAAlgorithm1(BaseAlgorithm):
        pca_components: int = 6
        normalize: bool = False

        def fit(self, train_df:pd.DataFrame): 
            data = train_df.sort_values(['year', 'week_no'])[['pid', 'date', 'emission']].pivot(index='date', columns='pid', values='emission')

            if self.normalize:
                mean_values = train_df.groupby('pid').emission.mean().to_numpy()
                data = data / mean_values
                data = data.fillna(0)
            else:
                mean_values = 1

            pca = PCA(n_components = self.pca_components, whiten = True)
            data_t = pca.fit_transform(data.to_numpy())

            data_pca = np.mean(data_t.reshape((-1, 53, self.pca_components)), axis=0)
            self.pca_pred = pca.inverse_transform(data_pca)
            self.pca_pred = np.multiply(self.pca_pred, mean_values)


        def predict(self, test_df:pd.DataFrame):
            return [self.pca_pred[int((row.date % 1)*100), row.pid] for _, row in test_df.iterrows()]



    proc = Preprocessor([TRAIN_CSV, TEST_CSV])
    train_df, test_df = proc.get_dfs()

    pl = Pipeline2021(train_df = train_df, 
                  test_df = test_df, 
                  target = 'emission', 
                  metrics = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
                  algorithm = PCAAlgorithm1(pca_components=6, normalize=False)
                 )

    pl.prepare_cv()
    pl.cv()

    submit_df = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')
    submit_df.emission = pl.predict()
    submit_df.to_csv('submission_pca1.csv', index=False)

    !head submission_pca1.csv
    
# Train results=[17.400447992446068, 13.860394893214197, 19.70961734612705]	Val results=[23.762264618054928, 30.222470534371666, 16.496753769909112]	Mean = 23.49 ± 5.61

In [6]:
if False:
    pca_components = 6

    fig, ax = plt.subplots(nrows=2, ncols=pca_components)
    fig.set_size_inches(30, 10)

    data = train_df.sort_values(['year', 'week_no'])[['pid', 'date', 'emission']].pivot(index='date', columns='pid', values='emission')

    pcas = (PCA(n_components=pca_components, whiten=True), PCA(n_components=pca_components, whiten=True))
    data1 = pcas[0].fit_transform(data.to_numpy())

    mean_values = train_df.groupby('pid').emission.mean()
    data = data / mean_values
    data = data.fillna(0)

    data2 = pcas[1].fit_transform(data.to_numpy())

    for m, pca in enumerate(pcas):
        for n in range(pca.n_components_):
            p = ax[m, n].tripcolor(proc.coords[:, 0], proc.coords[:, 1], pca.components_[n])
            ax[m, n].set_title(f'PCA_{n} (Explained {pca.explained_variance_ratio_[n]:.3f})')
            ax[m, n].plot(proc.coords[:, 0], proc.coords[:, 1], 'w.', markersize=1)
            fig.colorbar(p, ax=ax[m, n])

# PCA2 Algorithm

In [7]:
if True:
    @dataclass
    class PCAAlgorithm2(BaseAlgorithm):
        reg: BaseEstimator
        x_pca_components: int = 2
        y_pca_components: int = 5

        whiten: bool = False
        use_all_data: bool = True

        def pca_reduction(self, df, column, pca_components):
            data = df.sort_values('date')[['pid', 'date', column]].pivot(index='date', columns='pid', values=column).fillna(0).to_numpy()
            pca = PCA(n_components = pca_components, whiten = self.whiten)
            data_transformed = pca.fit_transform(data)
            return pca, data_transformed


        def fit(self, train_df:pd.DataFrame): 
            self.columns = [col for col in train_df.columns if (col not in ('ID_LAT_LON_YEAR_WEEK', 'week_no')) & ('_' in col) & ('angle' not in col)]

            self.pca_emission, data_emission = self.pca_reduction(train_df, 'emission', self.y_pca_components)
            self.mean_emission = np.tile(np.mean(data_emission.reshape((-1, 53, self.y_pca_components)), axis = 0), [10, 1])

            self.pcas = {}
            data = []
            for column in self.columns:
                self.pcas[column], data_transformed = self.pca_reduction(train_df, column, self.x_pca_components)
                data.append(data_transformed)
            data.append(self.mean_emission[:data[0].shape[0], :])
            if not self.use_all_data:
                data = [data[-1]]
            data = np.concatenate(data, axis=1)

            self.regs = []
            for n in range(data_emission.shape[1]):
                reg = clone(self.reg)
                reg.fit(data, data_emission[:, n])
                self.regs.append(reg)


        def predict(self, test_df:pd.DataFrame):
            data = []
            for column in self.columns:
                dt = test_df.sort_values('date')[['pid', 'date', column]].pivot(index='date', columns='pid', values=column).fillna(0).to_numpy()

                data_transformed = self.pcas[column].transform(dt)
                data.append(data_transformed)
            data.append(self.mean_emission[:data[0].shape[0], :])
            if not self.use_all_data:
                data = [data[-1]]
            data = np.concatenate(data, axis=1)

            data_emission = [reg.predict(data) for reg in self.regs]
            data_emission = np.stack(data_emission, axis=1)
            self.pca_pred = self.pca_emission.inverse_transform(data_emission)

            return [self.pca_pred[row.week_no, row.pid] for _, row in test_df.iterrows()]


    proc = Preprocessor([TRAIN_CSV, TEST_CSV])
    train_df, test_df = proc.get_dfs()

    algorithm = PCAAlgorithm2(reg=Lasso(1e5))

    pl = Pipeline2021(train_df = train_df, 
                  test_df = test_df, 
                  target = 'emission', 
                  metrics = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
                  algorithm = algorithm
                 )

    pl.prepare_cv()
    #pl.cv()

    # If using only data = [data[-1]]
    # Train results=[20.21349667978781, 15.938638748036908, 20.47409658664885]	Val results=[22.01175795461681, 28.570374103625394, 22.457737596452105]	Mean = 24.35 ± 2.99

    # If using all data
    # Train results=[21.473538199760156, 16.822880214830896, 21.64011790074873]	Val results=[26.296124289279522, 29.566012563872942, 24.838323036590005]	Mean = 26.90 ± 1.98

In [8]:
if True:
    reg = XGBRegressor(n_estimators = 300, max_depth = 4, learning_rate = 0.01, subsample = 0.5)
    pl.algorithm = PCAAlgorithm2(reg=reg, y_pca_components=8, use_all_data=True)
    pl.cv()

# If using only data = [data[-1]] LB = 32.99
# Train results=[14.772877589451626, 10.431708039446768, 15.029053555985346]	Val results=[20.003894149566104, 26.78881632449643, 19.529596231359957]	Mean = 22.11 ± 3.32

# If using all data LB = 30.79
# Train results=[17.021493159113582, 12.275437078774036, 18.349682595468735]	Val results=[20.377598555024935, 26.948277636370044, 20.293793823631393]	Mean = 22.54 ± 3.12
# Add is_covid & is_lockdown LB = 29.72
# Train results=[16.849774005566104, 12.276671286446696, 16.57368345876184]	Val results=[17.91517733569031, 26.9496417264071, 19.86816651090668]	Mean = 21.58 ± 3.88

/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/opt/conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/opt/conda/lib/python3.10/site-packages/sklearn/decompo

Val results=[30.50433525005058, 28.99586680133778, 26.29923444148491, 19.86816651090668]	Mean = 26.42 ± 4.07


In [9]:
if True:
    emission = np.array(pl.predict())
    

    for mult in [1, 1.05, 1.06, 1.07, 1.08]:
        submit_df = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')
        submit_df.emission = emission * 1.07
        submit_df.loc[test_df['longitude']==29.321, 'emission'] = mult * train_df.loc[(train_df['year']==2021) & (train_df['week_no']<=48) & (train_df['longitude'] == 29.321), 'emission'].values
        out = f'submission_pca2_x1.07_29.321x{mult:.2f}.csv'
        submit_df.to_csv(out, index=False)
        !head {out}

ID_LAT_LON_YEAR_WEEK,emission
ID_-0.510_29.290_2022_00,4.225600456500926
ID_-0.510_29.290_2022_01,4.458608191920423
ID_-0.510_29.290_2022_02,4.493135033800828
ID_-0.510_29.290_2022_03,4.493326904312325
ID_-0.510_29.290_2022_04,4.537351006058987
ID_-0.510_29.290_2022_05,4.522386123175717
ID_-0.510_29.290_2022_06,4.485195647433133
ID_-0.510_29.290_2022_07,4.520021307765288
ID_-0.510_29.290_2022_08,4.538194857277172
ID_LAT_LON_YEAR_WEEK,emission
ID_-0.510_29.290_2022_00,4.225600456500926
ID_-0.510_29.290_2022_01,4.458608191920423
ID_-0.510_29.290_2022_02,4.493135033800828
ID_-0.510_29.290_2022_03,4.493326904312325
ID_-0.510_29.290_2022_04,4.537351006058987
ID_-0.510_29.290_2022_05,4.522386123175717
ID_-0.510_29.290_2022_06,4.485195647433133
ID_-0.510_29.290_2022_07,4.520021307765288
ID_-0.510_29.290_2022_08,4.538194857277172
ID_LAT_LON_YEAR_WEEK,emission
ID_-0.510_29.290_2022_00,4.225600456500926
ID_-0.510_29.290_2022_01,4.458608191920423
ID_-0.510_29.290_2022_02,4.493135033800828
ID_-0.5

# PCA_SARIMA Algorithm

In [10]:
if False:
    @dataclass
    class PCA_SARIMA_Algorithm(BaseAlgorithm):
        sarimax_parameters: List
        x_pca_components: int = 2
        y_pca_components: int = 5

        whiten: bool = False
        use_all_data: bool = True

        def pca_reduction(self, df, column, pca_components):
            data = df.sort_values('date')[['pid', 'date', column]].pivot(index='date', columns='pid', values=column).fillna(0).to_numpy()
            pca = PCA(n_components = pca_components, whiten = self.whiten)
            data_transformed = pca.fit_transform(data)
            return pca, data_transformed


        def fit(self, train_df:pd.DataFrame):
            assert(self.y_pca_components <= len(self.sarimax_parameters))

            self.columns = [col for col in train_df.columns if (col not in ('ID_LAT_LON_YEAR_WEEK', 'week_no')) & ('_' in col) & ('angle' not in col) & ('is' not in col)]

            self.pca_emission, data_emission = self.pca_reduction(train_df, 'emission', self.y_pca_components)

            self.models = []
            for n in range(self.y_pca_components):
                model = SARIMAX(endog=data_emission[:, n], **self.sarimax_parameters[n]).fit(disp=False)
                self.models.append(model)        

            """self.mean_emission = np.tile(np.mean(data_emission.reshape((-1, 53, self.y_pca_components)), axis = 0), [10, 1])

            self.pcas = {}
            data = []
            for column in self.columns:
                self.pcas[column], data_transformed = self.pca_reduction(train_df, column, self.x_pca_components)
                data.append(data_transformed)
            data.append(self.mean_emission[:data[0].shape[0], :])
            if not self.use_all_data:
                data = [data[-1]]
            data = np.concatenate(data, axis=1)

            self.regs = []
            for n in range(data_emission.shape[1]):
                reg = clone(self.reg)
                reg.fit(data, data_emission[:, n])
                self.regs.append(reg)"""


        def predict(self, test_df:pd.DataFrame):
            data_emission = [model.forecast(len(test_df.date.unique())) for model in self.models]


            """return
            data = []
            for column in self.columns:
                dt = test_df.sort_values('date')[['pid', 'date', column]].pivot(index='date', columns='pid', values=column).fillna(0).to_numpy()

                data_transformed = self.pcas[column].transform(dt)
                data.append(data_transformed)
            data.append(self.mean_emission[:data[0].shape[0], :])
            if not self.use_all_data:
                data = [data[-1]]
            data = np.concatenate(data, axis=1)"""

            #data_emission = [reg.predict(data) for reg in self.regs]
            data_emission = np.stack(data_emission, axis=1)

            self.pca_pred = self.pca_emission.inverse_transform(data_emission)

            return [self.pca_pred[row.week_no, row.pid] for _, row in test_df.iterrows()]


    proc = Preprocessor([TRAIN_CSV, TEST_CSV])
    train_df, test_df = proc.get_dfs()

    sarimax_parameters = [
        {'order': (0, 0, 0), 'seasonal_order': (0, 1, 3, 53)},
        {'order': (1, 0, 2), 'seasonal_order': (1, 1, 0, 53)},
        {'order': (1, 0, 1), 'seasonal_order': (1, 1, 0, 53)},
        {'order': (2, 0, 3), 'seasonal_order': (1, 1, 0, 53)},
        {'order': (1, 0, 2), 'seasonal_order': (1, 1, 0, 53)},
        {'order': (2, 1, 0), 'seasonal_order': (1, 1, 0, 53)},
    ]

    algorithm = PCA_SARIMA_Algorithm(sarimax_parameters=sarimax_parameters)

    pl = Pipeline2021(train_df = train_df, 
                  test_df = test_df, 
                  target = 'emission', 
                  metrics = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
                  algorithm = algorithm
                 )

    pl.prepare_cv()
    pl.cv()

In [11]:
if False:
    submit_df = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')
    submit_df.emission = pl.predict()
    submit_df.to_csv('submission_pca_sarima.csv', index=False)

    !head submission_pca_sarima.csv

In [12]:
if False:
    def pca_reduction(df, column, pca_components):
        data = df.sort_values('date')[['pid', 'date', column]].pivot(index='date', columns='pid', values=column).fillna(0).to_numpy()
        pca = PCA(n_components = pca_components, whiten = False)
        data_transformed = pca.fit_transform(data)
        return pca, data_transformed

    def err(pred, gt):
        return np.sqrt(np.sum((pred - gt)**2) / np.sum(gt**2))

    _, de = pca_reduction(train_df, 'emission', 20)

    for n in range(5):
        pred1 = algorithm.models[n].forecast(53)
        pred2 = de[:, n].reshape((3, -1))[:2].mean(axis=0)
        gt = de[53*2:53*3, n]

        plt.figure()
        plt.plot(de[:, n])
        plt.plot(range(53*2, 53*3), pred1)

        print('\n\n=> ', err(pred1, gt), err(pred2, gt))

In [13]:
if False:
    _, de = pca_reduction(train_df, 'emission', 20)

    for n in [5]:
        fig, ax = plt.subplots(ncols=2, nrows=2)
        fig.set_size_inches(24,10)
        for m in range(3):
            ax[0, 0].plot(de[m*53:(m+1)*53, n], label=f'{2019+m}')
        ax[0, 0].legend()
        ax[0, 1].plot(de[:, n])

        sm.graphics.tsa.plot_acf(de[:, n], lags=78, ax=ax[1, 0])
        sm.graphics.tsa.plot_pacf(de[:, n], lags=78, ax=ax[1, 1])

        break

In [14]:
if False:
    proc = Preprocessor([TRAIN_CSV, TEST_CSV])
    train_df, test_df = proc.get_dfs()
    _, de = pca_reduction(train_df, 'emission', 20)
    _, de2 = pca_reduction(train_df[train_df.year != 2021], 'emission', 6)

    n = 5

    sarimax_parameters[n] = {'order': (1, 0, 1), 'seasonal_order': (1, 1, 0, 53)}
    model = SARIMAX(endog=de2[:2*53, n], **sarimax_parameters[n])
    pred1 = model.fit(disp=False).forecast(53)
    #pred1 = algorithm.models[0].forecast(53)
    pred2 = de[:, n].reshape((3, -1))[:2].mean(axis=0)
    gt = de[53*2:53*3, n]

    plt.plot(de[:, n], 'b')
    plt.plot(de2[:, n], 'r--')
    plt.plot(range(53*2, 53*3), pred1, 'r', linewidth=2)

    print('\n\n=> ', err(pred1, gt), err(pred2, gt))

In [15]:
if False:
    proc = Preprocessor([TRAIN_CSV, TEST_CSV])
    train_df, test_df = proc.get_dfs()
    _, de = pca_reduction(train_df, 'emission', 20)
    _, de2 = pca_reduction(train_df[train_df.year != 2021], 'emission', 6)

    n = 0

    model = SARIMAX(endog=de2[:2*53, n], **sarimax_parameters[n]).fit(disp=False)
    pred1 = model.forecast(53)
    #pred1 = algorithm.models[0].forecast(53)
    pred2 = de[:, n].reshape((3, -1))[:2].mean(axis=0)
    gt = de[53*2:53*3, n]

    plt.plot(de[:, n], 'b')
    plt.plot(de2[:, n], 'r--')
    plt.plot(range(53*2, 53*3), pred1, 'r', linewidth=2)

    print('\n\n=> ', err(pred1, gt), err(pred2, gt))